In [1]:
from general_tools.notebook import gpu_utils
GPU = 2
gpu_utils.setup_one_gpu(GPU)
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)

Picking GPU 2


In [2]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf

from tf_lab.in_out.basics import read_saved_epochs
                                                  
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.data_sets.model_net import pc_loader, classes_to_integers
from tf_lab.nips.helper import average_per_class, pclouds_with_zero_mean_in_unit_sphere

from sklearn.svm import LinearSVC
from general_tools.simpletons import sort_dict_by_key, sort_dict_by_val

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_samples = 2048
put_in_usphere = True
model_net = '10'
loss = 'emd'

In [5]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/'
experiment_name = 'no_conv_deeper_snc_rotated2048pts_' + loss

In [6]:
train_dir = osp.join(top_data_dir, experiment_name)
conf = Conf.load(osp.join(train_dir, 'configuration'))

if conf.train_dir != train_dir: # added to address moving training-folder to other location
    conf.train_dir = train_dir
    conf.save(osp.join(conf.train_dir, 'configuration'))

print conf
conf.n_output = conf.n_input
conf.allow_gpu_growth = True

              allow_gpu_growth: False
                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'b_norm': True, 'layer_sizes': [1024, 2048, 6144]}
                       encoder: encoder_with_convs_and_symmetry_new
                  encoder_args: {'filter_sizes': [1, 1, 1, 1], 'n_filters': [128, 128, 256, 512], 'b_norm': True, 'strides': [1, 1, 1, 1]}
               experiment_name: no_conv_deeper_snc_rotated2048pts_emd
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: emd
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 10
                     trai

In [7]:
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)
saved_epochs = read_saved_epochs(conf.train_dir)
last_epoch = saved_epochs[-1]
last_epoch = 1100
ae.restore_model(conf.train_dir, last_epoch, verbose=True)

Model restored in epoch 1100.


In [8]:
model_net_dir = '/orions4-zfs/projects/optas/DATA/Point_Clouds/Model_Net_' + model_net + '/from_manifold/'
model_net_dir = osp.join(model_net_dir, str(n_pc_samples))

search_pattern = '(.*)train(.*)\.ply$'
train_pc_files = sorted([f for f in files_in_subdirs(model_net_dir, search_pattern)])

search_pattern = '(.*)test(.*)\.ply$'
test_pc_files = sorted([f for f in files_in_subdirs(model_net_dir, search_pattern)])

In [9]:
verbose = True
n_threads = 22

pc, model_names, labels = load_point_clouds_from_filenames(train_pc_files, n_threads, pc_loader, verbose)

if put_in_usphere:
    pc = pclouds_with_zero_mean_in_unit_sphere(pc)

train_data = PointCloudDataSet(pc, labels=labels, init_shuffle=False)

pc_, model_names_, labels_ = load_point_clouds_from_filenames(test_pc_files, n_threads, pc_loader, verbose)

if put_in_usphere:
    pc_ = pclouds_with_zero_mean_in_unit_sphere(pc_)

test_data = PointCloudDataSet(pc_, labels=labels_, init_shuffle=False)

3991 pclouds were loaded. They belong in 10 shape-classes.
908 pclouds were loaded. They belong in 10 shape-classes.


In [10]:
train_feed, train_latent, train_classes = ae.embedding_at_tensor(train_data, conf, tensor_name='bottleneck')
cids = classes_to_integers(int(model_net), train_classes)[1]

test_feed, test_latent, test_classes = ae.embedding_at_tensor(test_data, conf, tensor_name='bottleneck')
cids_ = classes_to_integers(int(model_net), test_classes)[1]

In [11]:
opt_params_for_m40_chamfer = [0.25, 0.4, False, 'squared_hinge']

In [12]:
all_random_states = dict()
all_scores = dict()

intercept_scaling_space = [0.1, 0.2, 0.4, 0.5, 1, 2, 3, 4]

c_space = [0.01, 0.02, 0.05, 0.07, 0.08, 0.09,
           0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 
           0.45, 0.50]

dual_space = [False, True]

loss_space = ['hinge', 'squared_hinge']

In [17]:
for c in c_space:
    for intercept in intercept_scaling_space:
        for dual in dual_space:
            for loss in loss_space:
                try:
                    rs = np.random.get_state()
                    lsvc = LinearSVC(C=c, loss=loss, intercept_scaling=intercept, dual=dual, random_state=42)
                    lsvc.fit(train_latent, cids)
                    
                    print c, intercept, dual, loss
                    score = average_per_class(lsvc, test_latent, cids_)

                    print score
                    key = (c, intercept, dual, loss)
                    all_scores[key] = score
                    all_random_states[key] = rs
                except:
                    continue

0.01 0.1 False squared_hinge
0.94911627907
0.01 0.1 True hinge
0.947441860465
0.01 0.1 True squared_hinge
0.94911627907
0.01 0.2 False squared_hinge
0.94911627907
0.01 0.2 True hinge
0.945279069767
0.01 0.2 True squared_hinge
0.94911627907
0.01 0.4 False squared_hinge
0.94911627907
0.01 0.4 True hinge
0.946441860465
0.01 0.4 True squared_hinge
0.94911627907
0.01 0.5 False squared_hinge
0.94911627907
0.01 0.5 True hinge
0.946279069767
0.01 0.5 True squared_hinge
0.94911627907
0.01 1 False squared_hinge
0.94911627907
0.01 1 True hinge
0.947441860465
0.01 1 True squared_hinge
0.94911627907
0.01 2 False squared_hinge
0.94911627907
0.01 2 True hinge
0.947441860465
0.01 2 True squared_hinge
0.94911627907
0.01 3 False squared_hinge
0.94911627907
0.01 3 True hinge
0.947441860465
0.01 3 True squared_hinge
0.94911627907
0.01 4 False squared_hinge
0.94911627907
0.01 4 True hinge
0.946279069767
0.01 4 True squared_hinge
0.94911627907
0.02 0.1 False squared_hinge
0.954604651163
0.02 0.1 True hinge


In [21]:
best_responses = sort_dict_by_val(all_scores, reverse=True)
best_responses

[((0.15, 0.1, True, 'hinge'), 0.95909302325581403),
 ((0.08, 0.4, True, 'hinge'), 0.95709302325581402),
 ((0.09, 1, True, 'squared_hinge'), 0.95709302325581391),
 ((0.2, 0.1, True, 'squared_hinge'), 0.95676744186046514),
 ((0.4, 1, True, 'hinge'), 0.95593023255813958),
 ((0.2, 0.4, True, 'hinge'), 0.95593023255813958),
 ((0.08, 0.2, True, 'squared_hinge'), 0.95593023255813958),
 ((0.4, 3, True, 'hinge'), 0.9556046511627907),
 ((0.08, 1, True, 'squared_hinge'), 0.95493023255813958),
 ((0.07, 0.1, True, 'squared_hinge'), 0.95476744186046525),
 ((0.05, 4, True, 'squared_hinge'), 0.95476744186046503),
 ((0.02, 0.2, True, 'squared_hinge'), 0.9546046511627907),
 ((0.35, 0.5, True, 'squared_hinge'), 0.9546046511627907),
 ((0.02, 3, False, 'squared_hinge'), 0.9546046511627907),
 ((0.02, 0.1, True, 'squared_hinge'), 0.9546046511627907),
 ((0.02, 0.5, False, 'squared_hinge'), 0.9546046511627907),
 ((0.02, 0.1, False, 'squared_hinge'), 0.9546046511627907),
 ((0.02, 4, False, 'squared_hinge'), 0.9

In [53]:
c, intercept, dual, loss = best_responses[rank][0]
print c, intercept, dual, loss

0.15 0.1 True hinge


In [51]:
rank = 0
c, intercept, dual, loss = best_responses[rank][0]
# c, intercept, dual, loss = [0.02, 3, False, "squared_hinge"]
for i in range(30):
    print c, intercept, dual, loss
    lsvc = LinearSVC(C=c, loss=loss, intercept_scaling=intercept, dual=dual, random_state=42)
    lsvc.fit(train_latent, cids)
    score = average_per_class(lsvc, test_latent, cids_)
    print score

0.15 0.1 True hinge
0.95376744186
0.15 0.1 True hinge
0.947627906977
0.15 0.1 True hinge
0.947790697674
0.15 0.1 True hinge
0.950441860465
0.15 0.1 True hinge
0.890953488372
0.15 0.1 True hinge
0.948279069767
0.15 0.1 True hinge
0.935813953488
0.15 0.1 True hinge
0.944302325581
0.15 0.1 True hinge
0.955930232558
0.15 0.1 True hinge
0.948953488372
0.15 0.1 True hinge
0.936488372093
0.15 0.1 True hinge
0.942627906977
0.15 0.1 True hinge
0.946790697674
0.15 0.1 True hinge
0.945790697674
0.15 0.1 True hinge
0.94911627907
0.15 0.1 True hinge
0.914069767442
0.15 0.1 True hinge
0.933837209302
0.15 0.1 True hinge
0.929186046512
0.15 0.1 True hinge
0.952604651163
0.15 0.1 True hinge
0.943465116279
0.15 0.1 True hinge
0.946953488372
0.15 0.1 True hinge
0.951279069767
0.15 0.1 True hinge
0.95011627907
0.15 0.1 True hinge
0.935325581395
0.15 0.1 True hinge
0.923046511628
0.15 0.1 True hinge
0.951279069767
0.15 0.1 True hinge
0.943139534884
0.15 0.1 True hinge
0.953604651163
0.15 0.1 True hinge
0.9

{(0.01, 0.1, False, 'squared_hinge'): 0.94911627906976737,
 (0.01, 0.1, True, 'hinge'): 0.94744186046511625,
 (0.01, 0.1, True, 'squared_hinge'): 0.94911627906976737,
 (0.01, 0.2, False, 'squared_hinge'): 0.94911627906976737,
 (0.01, 0.2, True, 'hinge'): 0.94527906976744203,
 (0.01, 0.2, True, 'squared_hinge'): 0.94911627906976737,
 (0.01, 0.4, False, 'squared_hinge'): 0.94911627906976737,
 (0.01, 0.4, True, 'hinge'): 0.94644186046511636,
 (0.01, 0.4, True, 'squared_hinge'): 0.94911627906976737,
 (0.01, 0.5, False, 'squared_hinge'): 0.94911627906976737,
 (0.01, 0.5, True, 'hinge'): 0.94627906976744192,
 (0.01, 0.5, True, 'squared_hinge'): 0.94911627906976737,
 (0.01, 1, False, 'squared_hinge'): 0.94911627906976737,
 (0.01, 1, True, 'hinge'): 0.94744186046511625,
 (0.01, 1, True, 'squared_hinge'): 0.94911627906976737,
 (0.01, 2, False, 'squared_hinge'): 0.94911627906976737,
 (0.01, 2, True, 'hinge'): 0.94744186046511625,
 (0.01, 2, True, 'squared_hinge'): 0.94911627906976737,
 (0.01, 3,

In [ ]:
# New stuff for ICLR
# 1.  Chamfer
# encoder_args: {'filter_sizes': [1, 1, 1, 1, 1], 'strides': [1, 1, 1, 1, 1], 'n_filters': [64, 128, 128, 256, 512], 'dropout_prob': [0, 0, 0, 0, 0.5], 'b_norm': True}
# decoder_args: {'b_norm': True, 'layer_sizes': [512, 1024, 6144]}
# -> @ 980 epochs of training: best was on model-net40:
# 0.6 1 True hinge
# 0.840083537332
# -> @ 1620 epochs of training: best was on model-net40:
# 0.4 1 False squared_hinge
# 0.843783353733
# at @ 2000 epochs of training: best was on model-net40:
# (0.3, 0.2, False, 'squared_hinge'), 0.84087423500612002)


# FROM NIPS
# USING ALL ShapeNetCore DATA : ~57K models.
# Training at (max) 2K epochs.

# EMD, 2K, model_net_40
# 1.0 3 False squared_hinge
# 0.844814259486
# 1.0 3 True squared_hinge
# 0.844814259486

# EMD, 2K, model_net_10
# 0.4 2 True hinge
# 0.953279069767


# Chamfer, 2K model_net_40
# 0.3 4 False squared_hinge
# 0.856074051408
# 0.3 4 True squared_hinge
# 0.856074051408
# 0.6 1 False squared_hinge
# 0.856264075887
# 0.3 4 False squared_hinge
# 0.856074051408
# 0.6 1 False squared_hinge
# 0.856264075887
# 0.6 1 True squared_hinge
# 0.856264075887
# 0.7 1 False squared_hinge
# 0.856764075887

# Chamfer, 2K model_net_10
# 0.9 4 True hinge
# 0.950790697674
# 0.950790697674
# 0.8 0.5 False squared_hinge

# 1.0 4 True hinge
# 0.950790697674
# 1.0 3 True hinge
# 0.950790697674
# 0.9 4 True hinge
# 0.950790697674

# 0.7 4 True hinge
# 0.950790697674


# With WU 7 Classes:

# M10, CD, 2K epochs
# 0.4 4 True hinge
# 0.95176744186

# M40, CD, 2K epochs
# 1.0 1 True hinge
# 0.84410495716

# M10, EMD, ~2.5K epochs
# about 0.94

# M40, EMD, ~2.5K epochs
# about 0.83

In [ ]:
# all_flower_pots = test_feed[np.array(cids_) == 15]
# for i in range(len(all_flower_pots)):
#     Point_Cloud(all_flower_pots[i]).plot()
    
# y_pred = lsvc.predict(test_latent)
# gt_labels = np.array(cids_)

# scores_per_class = []

# for c in np.unique(gt_labels):
#     if c == 15:
#         scores_per_class.append(0.5)    
#         continue
#     index_c = gt_labels == c
#     n_class = float(np.sum(index_c))
#     s = np.sum(gt_labels[index_c] == y_pred[index_c])
#     s /= n_class
#     scores_per_class.append(s)
# print np.mean(scores_per_class)